In [92]:
import pandas as pd, spacy, random, pickle
import numpy as np, os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

def gen_data_frames(df, genre2int, max_num):

    nr_training_examples = {'Rock'   : 0,
                        'Pop'        : 0,
                        'Metal'      : 0,
                        'Country'    : 0,
                        'Hip-Hop'    : 0,
                        }

    '''nr_training_examples = {'Rock'   : 0,
                        'Pop'        : 0,
                        'R&B'        : 0,
                        'Metal'      : 0,
                        'Country'    : 0,
                        'Folk'       : 0,
                        'Hip-Hop'    : 0,
                        'Indie'      : 0,
                        'Jazz'       : 0,
                        'Electronic' : 0
                        }'''

    index = []
    lyrics = []
    labels = []
    index = 0
    list_of_genres = list(nr_training_examples.keys())
    for i in df['genre']:

        if i in list_of_genres and nr_training_examples[i] != max_num:
            nr_training_examples[i] += 1
            lyrics.append(df['lyrics'][index])
            labels.append(genre2int[i])
            index += 1
        else:
            index += 1
            continue

    df = pd.DataFrame(data=zip(lyrics,labels), columns=['lyrics', 'genre'])

    x = lyrics
    y = labels
    return df, x, y

def get_stopwords(str_stopwords, punctuation):
    str_stopwords += ['/n']
    str_stopwords += punctuation
    return str_stopwords

def tokenize(text_data, stopwords):
    training_data = []
    index = 0
    tokenized = []

    for i in tqdm(text_data):

        tokens = word_tokenize(i.lower())
        for x in tokens:
            if x in stopwords:
                tokens.remove(x)

            else:
                continue
        tokenized.append(tokens)

    return tokenized

def lemmatized(text_data):
    nlp = spacy.load('en_core_web_sm')
    lemmatized_data = []
    for i in tqdm(text_data):

        doc = nlp(' '.join(i))
        lemmatized = []
        for word in doc:
            lemmatized.append(word.lemma_)
        lemmatized_data.append(lemmatized)
    return lemmatized_data

def output_csv(data, labels, path):
    untokenized = []
    for i in data:
        untokenized.append(' '.join(i))
    df = pd.DataFrame(data=zip(untokenized, labels), columns=['lyrics', 'genre'])
    df.to_csv(path)
    return df

def output_txt(x, y, x_path, y_path):

    directory = 'data/pre_processed/'

    if os.path.exists(directory) == False:
        os.mkdir(directory)

    with open('{}{}'.format(directory, x_path), 'w') as file:
        for i in x:
            file.write(' '.join(i))
            file.write('\n')

    with open('{}{}'.format(directory,y_path), 'w') as file:
        for i in y:
            file.write(str(i))
            file.write('\n')

def output_dl(data, filename):
    with open('dataloaders/' + filename, 'wb') as file:
        pickle.dump(data, file)

In [93]:
train_lyrics = pd.read_csv('data/raw/train.csv')
test_lyrics = pd.read_csv('data/raw/test.csv')
genres = ['Rock', 'Pop', 'Metal', 'Country',
  'Hip-Hop']
'''genres = ['Rock', 'Pop', 'R&B', 'Metal', 'Country', 'Folk',
  'Hip-Hop', 'Indie', 'Jazz', 'Electronic']'''
int2genre = dict(enumerate(genres))
genre2int = {genre : num for num, genre in int2genre.items()}
print('Trimming datast')
training_df, x_train, y_train = gen_data_frames(train_lyrics, genre2int, 100)
testing_df, x_test, y_test = gen_data_frames(test_lyrics, genre2int, 50)
stops = get_stopwords(stopwords.words('english'), punctuation)
print('Tokenizing')
train_tokenized = tokenize(x_train, stops)
output_txt(train_tokenized, y_train, 'prac_x.txt', 'prac_y.txt')
test_tokenized = tokenize(x_test, stops)
output_txt(test_tokenized, y_test, 'x_test.txt', 'y_test.txt')


 11%|█▏        | 57/500 [00:00<00:00, 567.66it/s]

Trimming datast
Tokenizing



100%|██████████| 500/500 [00:00<00:00, 568.27it/s]

100%|██████████| 250/250 [00:00<00:00, 574.36it/s]


In [3]:
from ProjDataset import ProjDataset
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch, pickle, random, os
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
import torch.optim as optim
from torch import nn
from GRUNetwork import RNN_GRU
import torch, spacy

nlp = spacy.load('en_core_web_md')

def load_data(txt, labels):
    x = []
    with open(txt, 'r') as file:
        for i in file:
            x.append(i.split(' ')[:-1])
    y = []

    with open(labels, 'r') as file:
        for i in file:
            y.append(int(i))
    return x,y

def get_vocab(text_data):
    vocab = []
    for i in text_data:
        vocab += [x for x in i]
    return vocab

def encode(text_data,vocab):
    text_encoded = []
    for i in tqdm(text_data):
        encoded = []
        for word in i[:100]:
            if word in vocab:
                encoded.append(vocab[word])
            else:

                encoded.append(random.randint(0,len(vocab)))
        text_encoded.append(torch.LongTensor(encoded))

    return pad_sequence(text_encoded, batch_first=True, padding_value=0)

def output_dl(data, filename):
    with open('dataloaders/' + filename, 'wb') as file:
        pickle.dump(data, file)

def train(model, train_dl, vocab_size, device, nr_of_epochs, batch_size, hidden_size):

    print('Training')
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)
    model.train()
    model = model.to(device)
    epoch_nr = 0
    EPOCH = list(range(nr_of_epochs))
    tenp = round(len(train_dl,) / 10)
    avg_loss = 0 
    for epoch in tqdm(EPOCH):
        epoch_nr += 1
        epoch_loss = []
        h = model.init_hidden(hidden_size)
        count = 0
        percent = 0
        for (x,y) in tqdm(train_dl):
            if len(x) != batch_size:
                break
                
            else:
                
                count +=1
                x = x.to(device)
                y = y.to(device)
                optimizer.zero_grad()
                h = h.data
                out, h = model(x, h, device)
                loss = criterion(out, y.long())

                loss.backward()
                epoch_loss.append(loss.item())
                optimizer.step()

                avg_loss = sum(epoch_loss) / len(epoch_loss)
        print("Average loss at epoch %d: %.7f" % (epoch_nr, avg_loss))
    return model

In [95]:
x_train, y_train = load_data('data/pre_processed/prac_x.txt',
                                 'data/pre_processed/prac_y.txt')
x_test, y_test = load_data('data/pre_processed/x_test.txt',
                             'data/pre_processed/y_test.txt')

train_vocab = get_vocab(x_train)
int2wor = dict(enumerate(set(train_vocab)))
wor2int = {a : b for b, a in int2wor.items()}

In [97]:
x_train_embeddings = []
for i in tqdm(x_train):
    x_train_embeddings.append(nlp(' '.join(i)).vector)


100%|██████████| 500/500 [00:13<00:00, 35.93it/s]


In [96]:
len(wor2int)

11970

In [98]:
emb_vocab = []
for i in tqdm(wor2int):
    emb_vocab.append(nlp(i).vector)


 20%|██        | 2437/11970 [00:15<01:00, 156.65it/s]


 41%|████      | 4864/11970 [00:30<00:43, 163.29it/s]


 61%|██████    | 7292/11970 [00:44<00:28, 163.75it/s]


 81%|████████  | 9715/11970 [00:59<00:13, 162.02it/s]


100%|██████████| 11970/11970 [01:13<00:00, 161.83it/s]


In [34]:
train_emb_dataset = ProjDataset(x_train_embeddings, y_train)

In [35]:
traindl_emb = DataLoader(train_emb_dataset, batch_size=200, shuffle=True)

In [56]:
train_dataset = ProjDataset(x_train, y_train)
test_dataset = ProjDataset(x_test, y_test)
train_dl = DataLoader(train_dataset, batch_size=200, shuffle=True)
nlp = spacy.load('en_core_web_md')

In [69]:
from torch import nn
import torch, spacy



class RNN_GRU(nn.Module):
    def __init__(self, vocab_size, seq_len, input_size, hidden_size, num_layers, output_size, device, dropout=0.0, pretrained=bool):
        super().__init__()
        self.num_layers = num_layers
        self.seq_len = seq_len
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = device
        self.pretrained = pretrained
        self.emb = nn.Embedding(vocab_size, input_size).to(device)
        self.gru = nn.GRU(input_size, hidden_size,
                          num_layers=self.num_layers, batch_first=True, dropout=dropout).to(device)
        self.fc = nn.Linear(hidden_size * seq_len, output_size).to(device)

    def forward(self, sequence, hidden_layer, device):
        if self.pretrained == True:
            output = sequence
        else:
            output = self.emb(sequence).to(device)
        
        hidden_layer = hidden_layer.to(self.device)
        output, hidden_layer = self.gru(output.unsqueeze(0), hidden_layer)
        output = output.contiguous().view(-1, self.hidden_size *
                                          len(sequence[0]))
        output = self.fc(output).to(device)

        return output, hidden_layer

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size).float().to(self.device)



In [70]:
model = RNN_GRU(vocab_size = len(wor2int), seq_len=300, input_size=300,
               hidden_size = 128, num_layers = 2, output_size = 5,
               device = 'cuda:01', dropout=0.01, pretrained=True)

In [71]:
device = 'cuda:01'
batch_size = 200

In [76]:
print('Training')
nr_of_epochs = 1
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
model.train()
model = model.to(device)
epoch_nr = 0
EPOCH = list(range(nr_of_epochs))
tenp = round(len(traindl_emb,) / 10)
avg_loss = 0 
for epoch in tqdm(EPOCH):
    epoch_nr += 1
    epoch_loss = []
    h = model.init_hidden(200)
    count = 0
    percent = 0
    for (x,y) in tqdm(traindl_emb):
        if len(x) != batch_size:
            break

        else:
            
            count +=1
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            h = h.data
            
            out, h = model(x, h, device)
            loss = criterion(out, y.long())

            loss.backward()
            epoch_loss.append(loss.item())
            optimizer.step()

        
            avg_loss = sum(epoch_loss) / len(epoch_loss)
    break
    print("Average loss at epoch %d: %.7f" % (epoch_nr, avg_loss))

  0%|          | 0/1 [00:00<?, ?it/s]

Training


RuntimeError: Expected hidden size (2, 1, 128), got (2, 200, 128)